# Imports and configuration

We set the path to the config.cfg file using the environment variable 'JCMPYTHON_CONFIG_FILE'. If you do not have a configuration file yet, please look into the **Setting up a configuration file** example. 

In [ ]:
import os
os.environ['JCMPYTHON_CONFIG_FILE'] = '/path/to/your/config.cfg'

Now we can import `jcmpython` and `numpy`. Since the parent directory, which contains the jcmpython module, is not automatically in our path, we need to append it before.

In [ ]:
import sys
sys.path.append('..')
import jcmpython as jpy
import numpy as np

# Prepare

**Creating a `JCMProject`**:  The `mie2D`-project is located in a subdirectory of our *project catalog*, which is configured in the section `Data`, option `projects` in the configuration file.

In [ ]:
project = jpy.JCMProject('scattering/mie/mie2D')

**Set the keys** that are necessary to translate the JCM template files.

In [ ]:
mie_keys = {'constants' :{}, # <-- can be anything, but is not looped over and not stored in the HDF5 store
            'parameters': {}, # <-- everything that needs to be stored, but is not in layout.jcmt
            'geometry': {'radius':np.linspace(0.3, 0.5, 40)}} # <-- same as before, but layout.jcmt-relevant

**Initialize a `SimulationSet`**. We ignore the configured storage here and save everything in the current working dir.

In [ ]:
simuset = jpy.SimulationSet(project, mie_keys,
                            storage_base=os.path.abspath('tmp_storage_folder'))

**Make a schedule**: this checks which simulations need to be run and sorts them depending on their geometry.

In [ ]:
simuset.make_simulation_schedule()

**Define a processing function**: all post process results are passed to this function automatically. It *must* return a `dict` with your results.

In [ ]:
def read_scs(pp):
    results = {} #must be a dict
    results['SCS'] = pp[0]['ElectromagneticFieldEnergyFlux'][0][0].real
    return results

# Solve

**Run your simulations**. The results will be appended to the HDF5 store.

In [ ]:
simuset.run(processing_func=read_scs)

**Plot**

In [ ]:
%matplotlib inline
data = simuset.get_store_data().sort_values(by='radius')
data.plot(x='radius', y='SCS', title='Results of the simulation')

**Write the data to CSV**.

In [ ]:
simuset.write_store_data_to_file() # default is results.csv in the storage folder